# LSTM EFAS5: entrenamiento y validación
***

***Autor:** Jesús Casado Rodríguez*<br>
***Fecha:** 30-09-2023*<br>

**Introducción:**<br>
En este _notebook_ se analizan entrena y valida el modelo LSTM que replica el caudal simulado en EFAS5.

**Por hacer**:

In [ ]:
%%capture
! pip install neuralhydrology

In [ ]:
import pickle
import pandas as pd
from pathlib import Path
import torch
from tqdm.notebook import tqdm

from neuralhydrology.utils.config import Config
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/TFM/notebook/models/
from model_utils import *
%cd ./EFAS5/

### Configuración

In [ ]:
# cargar archivo de configuración
config_file = Path('config_test3.yml')
cfg = Config(config_file)

target = cfg.target_variables[0]

print(cfg.experiment_name)
print('epochs:', cfg.epochs, sep='\t\t')
print('hidden size:', cfg.hidden_size, sep='\t')
print('batch size:', cfg.batch_size, sep='\t')
print('dropout:', cfg.output_dropout, sep='\t')
print('clip gradients:', cfg.clip_gradient_norm, sep='\t')

### Entrenamiento

In [ ]:
# si se dispone de GPU
if torch.cuda.is_available():
    start_run(config_file=config_file)
# si sólo se dispone de CPU
else:
    start_run(config_file=config_file, gpu=-1)

### Simulación

In [ ]:
run_dir = Path(f'runs/{cfg.experiment_name}_1609_185822')

In [ ]:
# extraer el rendimiento de cada época y muestra
metrics = pd.DataFrame(columns=range(1, cfg.epochs + 1))
for epoch in tqdm(metrics.columns):
    try:
        _, metrics[epoch] = get_results(run_dir, 'validation', epoch=epoch)
    except:
        continue

best_epoch = metrics.median().idxmax()
print('mejor época: {0}\t\tKGE = {1:.3f}'.format(best_epoch, metrics.median().max()))

#### Periodo de entrenamiento

In [ ]:
period = 'train'

if run_all:
    for epoch in np.arange(1, cfg.epochs + 1):
        if torch.cuda.is_available(): # con GPU
            eval_run(run_dir=run_dir, period=period, epoch=epoch, gpu=0)
        else: # con CPU
            eval_run(run_dir=run_dir, period=period, epoch=epoch, gpu=-1)
else:
    if torch.cuda.is_available(): # con GPU
        eval_run(run_dir=run_dir, period=period, epoch=best_epoch, gpu=0)
    else: # con CPU
        eval_run(run_dir=run_dir, period=period, epoch=best_epoch, gpu=-1)

#### Periodo de evaluación

In [ ]:
period = 'test'

if run_all:
    for epoch in np.arange(1, cfg.epochs + 1):
        if torch.cuda.is_available(): # con GPU
            eval_run(run_dir=run_dir, period=period, epoch=epoch, gpu=0)
        else: # con CPU
            eval_run(run_dir=run_dir, period=period, epoch=epoch, gpu=-1)
else:
    if torch.cuda.is_available(): # con GPU
        eval_run(run_dir=run_dir, period=period, epoch=best_epoch, gpu=0)
    else: # con CPU
        eval_run(run_dir=run_dir, period=period, epoch=best_epoch, gpu=-1)